In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms
from PIL import Image

/home/marco/marco/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Reprendre la même architecture

In [2]:
class SiameseNetwork(nn.Module):
    def __init__(self, embedding_dim=128):
        super(SiameseNetwork, self).__init__()
        backbone = models.resnet18(pretrained=False)  # pas besoin de pretrained ici
        num_ftrs = backbone.fc.in_features
        backbone.fc = nn.Identity()
        self.backbone = backbone
        self.embedding = nn.Linear(num_ftrs, embedding_dim)
        self.classifier = nn.Sequential(
            nn.Linear(embedding_dim * 2, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward_once(self, x):
        feat = self.backbone(x)
        emb = self.embedding(feat)
        emb = F.normalize(emb, p=2, dim=1)
        return emb

    def forward(self, x1, x2):
        emb1 = self.forward_once(x1)
        emb2 = self.forward_once(x2)
        combined = torch.cat((emb1, emb2), dim=1)
        out = self.classifier(combined)
        return out

## 2. Charger le modèle sauvegardé

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SiameseNetwork().to(device)
model.load_state_dict(torch.load("siamese_resnet18_binary.pth", map_location=device))
model.eval()

/home/marco/marco/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/marco/marco/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


SiameseNetwork(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, t

## 3. Prétraitement des images

In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # même prétraitement que l’entraînement
])

def load_image(path):
    img = Image.open(path).convert("RGB")  # convert to 3 channels
    return transform(img).unsqueeze(0).to(device)  # ajouter batch dim

## 4. Fonction de prédiction

In [5]:
def verify_signature(path1, path2, threshold=0.5):
    img1 = load_image(path1)
    img2 = load_image(path2)

    with torch.no_grad():
        prob = model(img1, img2).item()

    if prob > threshold:
        print(f"✅ Même signataire (probabilité = {prob:.4f})")
    else:
        print(f"❌ Faux signataire (probabilité = {prob:.4f})")

## 5. Exemple d’utilisation

In [6]:
sig1 = "/home/marco/Projet_du_Fin_DEtude_M2/Datasets/full_org/original_1_5.png"
sig2 = "/home/marco/Projet_du_Fin_DEtude_M2/Datasets/full_forg/forgeries_5_3.png"

verify_signature(sig1, sig2)

✅ Même signataire (probabilité = 0.5079)


/home/marco/marco/lib/python3.11/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


In [7]:
# === Backbone pré-entraîné ===
class EmbeddingNet(nn.Module):
    def __init__(self):
        super().__init__()
        backbone = models.resnet18(pretrained=True)
        # retirer la dernière couche (classification)
        self.feature_extractor = nn.Sequential(*list(backbone.children())[:-1])
        self.fc = nn.Linear(backbone.fc.in_features, 128)  # vecteur 128D

    def forward(self, x):
        x = self.feature_extractor(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

In [8]:
# === Siamese Network ===
class SiameseNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding_net = EmbeddingNet()

    def forward(self, x1, x2):
        out1 = self.embedding_net(x1)
        out2 = self.embedding_net(x2)
        return out1, out2

In [9]:
# === Fonction Similarité ===
def contrastive_loss(out1, out2, label, margin=1.0):
    euclidean_distance = F.pairwise_distance(out1, out2)
    loss = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                      (label) * torch.pow(torch.clamp(margin - euclidean_distance, min=0.0), 2))
    return loss